In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install mido
import numpy as np
import tensorflow as tf
#physical_devices = tf.config.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
from tensorflow.keras.layers import Input, LSTM, Dropout, Dense, Flatten
from tensorflow.keras.models import Model

     |████████████████████████████████| 61kB 3.1MB/s 


In [ ]:
note_duration = np.load('/content/drive/MyDrive/Major Project/paper/midi_240_epochs.npy')
note_duration = note_duration.reshape(-1, 3).tolist()

In [ ]:
def duration_conversion(duration):
    if duration < 0.375:
        duration = 0.25
    elif duration >= 0.375 and duration < 0.625:
        duration = 0.5
    elif duration >= 0.625 and duration < 0.875:
        duration = 0.75
    elif duration >= 0.875 and duration < 1.25:
        duration = 1
    elif duration >= 1.25 and duration < 1.75:
        duration = 1.5
    elif duration >= 1.75 and duration < 2.5:
        duration = 2
    elif duration >= 2.5 and duration < 3.5:
        duration = 3
    elif duration >= 3.5 and duration <= 5:
        duration = 4
    elif duration > 5 and duration <= 7:
        duration = 6
    elif duration > 7 and duration <= 12:
        duration = 8
    elif duration > 12 and duration <= 24:
        duration = 16
    else:
        duration = 32
    return duration

In [ ]:
def rest_conversion(duration):
    if duration < 0.05:
        duration = 0
    elif duration >= 0.05 and duration < 0.375:
        duration = 0.25
    elif duration >= 0.375 and duration < 0.625:
        duration = 0.5
    elif duration >= 0.625 and duration < 0.875:
        duration = 0.75
    elif duration >= 0.875 and duration < 1.25:
        duration = 1
    elif duration >= 1.25 and duration < 1.75:
        duration = 1.5
    elif duration >= 1.75 and duration < 2.5:
        duration = 2
    elif duration >= 2.5 and duration < 3.5:
        duration = 3
    elif duration >= 3.5 and duration <= 5:
        duration = 4
    elif duration > 5 and duration <= 7:
        duration = 6
    elif duration > 7 and duration <= 12:
        duration = 8
    elif duration > 12 and duration <= 24:
        duration = 16
    else:
        duration = 32
    return duration

In [ ]:
bpm = 90
for a in note_duration:
    a[1] = duration_conversion(a[1])
    a[1] = a[1] * 60 / bpm
    a[2] = rest_conversion(a[2])
    a[2] = a[2] * 60 / bpm

In [ ]:
note_duration

[[66.98819732666016, 0.3333333333333333, 0.16666666666666666],
 [53.80523681640625, 0.3333333333333333, 0.16666666666666666],
 [79.4658432006836, 0.16666666666666666, 0.3333333333333333],
 [85.96997833251953, 0.16666666666666666, 0.16666666666666666],
 [86.12448120117188, 0.16666666666666666, 0.16666666666666666],
 [82.1078109741211, 0.16666666666666666, 0.16666666666666666],
 [67.1144790649414, 0.16666666666666666, 0.16666666666666666],
 [56.2026252746582, 0.16666666666666666, 0.0],
 [50.029937744140625, 0.16666666666666666, 0.16666666666666666],
 [68.41942596435547, 0.16666666666666666, 0.3333333333333333],
 [67.53046417236328, 0.16666666666666666, 0.16666666666666666],
 [66.6111831665039, 0.16666666666666666, 0.16666666666666666],
 [68.5562515258789, 0.16666666666666666, 0.16666666666666666],
 [70.23397064208984, 0.16666666666666666, 0.3333333333333333],
 [75.76205444335938, 0.16666666666666666, 0.3333333333333333],
 [62.920265197753906, 0.16666666666666666, 0.0],
 [65.9151916503906

In [ ]:
part_on = []
for idx, part in enumerate(note_duration):
    temp = []
    temp.append("note_on")
    temp.append(part[0])
    if idx == 0:
        temp.append(0)
    else:
        temp.append(part_on[idx - 1][2] + note_duration[idx - 1][2])
    part_on.append(temp)

In [ ]:
part_on

[['note_on', 66.98819732666016, 0],
 ['note_on', 53.80523681640625, 0.16666666666666666],
 ['note_on', 79.4658432006836, 0.3333333333333333],
 ['note_on', 85.96997833251953, 0.6666666666666666],
 ['note_on', 86.12448120117188, 0.8333333333333333],
 ['note_on', 82.1078109741211, 0.9999999999999999],
 ['note_on', 67.1144790649414, 1.1666666666666665],
 ['note_on', 56.2026252746582, 1.3333333333333333],
 ['note_on', 50.029937744140625, 1.3333333333333333],
 ['note_on', 68.41942596435547, 1.5],
 ['note_on', 67.53046417236328, 1.8333333333333333],
 ['note_on', 66.6111831665039, 2.0],
 ['note_on', 68.5562515258789, 2.1666666666666665],
 ['note_on', 70.23397064208984, 2.333333333333333],
 ['note_on', 75.76205444335938, 2.6666666666666665],
 ['note_on', 62.920265197753906, 3.0],
 ['note_on', 65.91519165039062, 3.0],
 ['note_on', 54.817840576171875, 3.3333333333333335],
 ['note_on', 73.86437225341797, 3.3333333333333335],
 ['note_on', 80.84966278076172, 3.666666666666667],
 ['note_on', 77.46525

In [ ]:
part_off = []
for idx, part in enumerate(note_duration):
    temp = []
    temp.append("note_off")
    temp.append(part[0])
    temp.append(part_on[idx][2] + note_duration[idx][1])
    part_off.append(temp)

In [ ]:
part_off

[['note_off', 66.98819732666016, 0.3333333333333333],
 ['note_off', 53.80523681640625, 0.5],
 ['note_off', 79.4658432006836, 0.5],
 ['note_off', 85.96997833251953, 0.8333333333333333],
 ['note_off', 86.12448120117188, 0.9999999999999999],
 ['note_off', 82.1078109741211, 1.1666666666666665],
 ['note_off', 67.1144790649414, 1.3333333333333333],
 ['note_off', 56.2026252746582, 1.5],
 ['note_off', 50.029937744140625, 1.5],
 ['note_off', 68.41942596435547, 1.6666666666666667],
 ['note_off', 67.53046417236328, 2.0],
 ['note_off', 66.6111831665039, 2.1666666666666665],
 ['note_off', 68.5562515258789, 2.333333333333333],
 ['note_off', 70.23397064208984, 2.4999999999999996],
 ['note_off', 75.76205444335938, 2.833333333333333],
 ['note_off', 62.920265197753906, 3.1666666666666665],
 ['note_off', 65.91519165039062, 3.1666666666666665],
 ['note_off', 54.817840576171875, 3.5],
 ['note_off', 73.86437225341797, 3.5],
 ['note_off', 80.84966278076172, 3.8333333333333335],
 ['note_off', 77.4652557373046

In [ ]:
total_notes = part_on + part_off
print(total_notes)

[['note_on', 66.98819732666016, 0], ['note_on', 53.80523681640625, 0.16666666666666666], ['note_on', 79.4658432006836, 0.3333333333333333], ['note_on', 85.96997833251953, 0.6666666666666666], ['note_on', 86.12448120117188, 0.8333333333333333], ['note_on', 82.1078109741211, 0.9999999999999999], ['note_on', 67.1144790649414, 1.1666666666666665], ['note_on', 56.2026252746582, 1.3333333333333333], ['note_on', 50.029937744140625, 1.3333333333333333], ['note_on', 68.41942596435547, 1.5], ['note_on', 67.53046417236328, 1.8333333333333333], ['note_on', 66.6111831665039, 2.0], ['note_on', 68.5562515258789, 2.1666666666666665], ['note_on', 70.23397064208984, 2.333333333333333], ['note_on', 75.76205444335938, 2.6666666666666665], ['note_on', 62.920265197753906, 3.0], ['note_on', 65.91519165039062, 3.0], ['note_on', 54.817840576171875, 3.3333333333333335], ['note_on', 73.86437225341797, 3.3333333333333335], ['note_on', 80.84966278076172, 3.666666666666667], ['note_on', 77.46525573730469, 4.0], ['n

In [ ]:
total_notes.sort(key = lambda x: x[2])

In [ ]:
total_notes

[['note_on', 66.98819732666016, 0],
 ['note_on', 53.80523681640625, 0.16666666666666666],
 ['note_on', 79.4658432006836, 0.3333333333333333],
 ['note_off', 66.98819732666016, 0.3333333333333333],
 ['note_off', 53.80523681640625, 0.5],
 ['note_off', 79.4658432006836, 0.5],
 ['note_on', 85.96997833251953, 0.6666666666666666],
 ['note_on', 86.12448120117188, 0.8333333333333333],
 ['note_off', 85.96997833251953, 0.8333333333333333],
 ['note_on', 82.1078109741211, 0.9999999999999999],
 ['note_off', 86.12448120117188, 0.9999999999999999],
 ['note_on', 67.1144790649414, 1.1666666666666665],
 ['note_off', 82.1078109741211, 1.1666666666666665],
 ['note_on', 56.2026252746582, 1.3333333333333333],
 ['note_on', 50.029937744140625, 1.3333333333333333],
 ['note_off', 67.1144790649414, 1.3333333333333333],
 ['note_on', 68.41942596435547, 1.5],
 ['note_off', 56.2026252746582, 1.5],
 ['note_off', 50.029937744140625, 1.5],
 ['note_off', 68.41942596435547, 1.6666666666666667],
 ['note_on', 67.53046417236

In [ ]:
total_notes[-1]

['note_off', 86.18614959716797, 22.333333333333336]

In [ ]:
for idx, part in enumerate(total_notes):
    if idx == (len(total_notes) - 1):
        part[2] = 0
    else:
        part[2] = total_notes[idx + 1][2] - part[2]
print(total_notes)   

[['note_on', 66.98819732666016, 0.16666666666666666], ['note_on', 53.80523681640625, 0.16666666666666666], ['note_on', 79.4658432006836, 0.0], ['note_off', 66.98819732666016, 0.16666666666666669], ['note_off', 53.80523681640625, 0.0], ['note_off', 79.4658432006836, 0.16666666666666663], ['note_on', 85.96997833251953, 0.16666666666666663], ['note_on', 86.12448120117188, 0.0], ['note_off', 85.96997833251953, 0.16666666666666663], ['note_on', 82.1078109741211, 0.0], ['note_off', 86.12448120117188, 0.16666666666666663], ['note_on', 67.1144790649414, 0.0], ['note_off', 82.1078109741211, 0.16666666666666674], ['note_on', 56.2026252746582, 0.0], ['note_on', 50.029937744140625, 0.0], ['note_off', 67.1144790649414, 0.16666666666666674], ['note_on', 68.41942596435547, 0.0], ['note_off', 56.2026252746582, 0.0], ['note_off', 50.029937744140625, 0.16666666666666674], ['note_off', 68.41942596435547, 0.16666666666666652], ['note_on', 67.53046417236328, 0.16666666666666674], ['note_on', 66.61118316650

In [ ]:
total_notes

[['note_on', 66.98819732666016, 0.16666666666666666],
 ['note_on', 53.80523681640625, 0.16666666666666666],
 ['note_on', 79.4658432006836, 0.0],
 ['note_off', 66.98819732666016, 0.16666666666666669],
 ['note_off', 53.80523681640625, 0.0],
 ['note_off', 79.4658432006836, 0.16666666666666663],
 ['note_on', 85.96997833251953, 0.16666666666666663],
 ['note_on', 86.12448120117188, 0.0],
 ['note_off', 85.96997833251953, 0.16666666666666663],
 ['note_on', 82.1078109741211, 0.0],
 ['note_off', 86.12448120117188, 0.16666666666666663],
 ['note_on', 67.1144790649414, 0.0],
 ['note_off', 82.1078109741211, 0.16666666666666674],
 ['note_on', 56.2026252746582, 0.0],
 ['note_on', 50.029937744140625, 0.0],
 ['note_off', 67.1144790649414, 0.16666666666666674],
 ['note_on', 68.41942596435547, 0.0],
 ['note_off', 56.2026252746582, 0.0],
 ['note_off', 50.029937744140625, 0.16666666666666674],
 ['note_off', 68.41942596435547, 0.16666666666666652],
 ['note_on', 67.53046417236328, 0.16666666666666674],
 ['not

In [ ]:
from mido import MidiFile, MidiTrack, Message
mid = MidiFile()
track = MidiTrack()
for i in range(100):
    # 147 means note_on
    # 67 is velocity
    if total_notes[i][0] == "note_on":
        note = np.asarray([147, int(total_notes[i][1]), 67])
        bytes = note.astype(int)
        msg = Message.from_bytes(bytes[0:3])
        time = total_notes[i][2]
        msg.time = round(time * mid.ticks_per_beat * 2)
    else:
        note = np.asarray([147, int(total_notes[i][1]), 0])
        bytes = note.astype(int)
        msg = Message.from_bytes(bytes[0:3])
        time = total_notes[i][2]
        msg.time = round(time * mid.ticks_per_beat * 2)
    track.append(msg)
mid.tracks.append(track)
mid.save('Paper.mid')

In [ ]:
for msg in mid:
  print(msg)

note_on channel=3 note=66 velocity=67 time=0.16666666666666666
note_on channel=3 note=53 velocity=67 time=0.16666666666666666
note_on channel=3 note=79 velocity=67 time=0
note_on channel=3 note=66 velocity=0 time=0.16666666666666666
note_on channel=3 note=53 velocity=0 time=0
note_on channel=3 note=79 velocity=0 time=0.16666666666666666
note_on channel=3 note=85 velocity=67 time=0.16666666666666666
note_on channel=3 note=86 velocity=67 time=0
note_on channel=3 note=85 velocity=0 time=0.16666666666666666
note_on channel=3 note=82 velocity=67 time=0
note_on channel=3 note=86 velocity=0 time=0.16666666666666666
note_on channel=3 note=67 velocity=67 time=0
note_on channel=3 note=82 velocity=0 time=0.16666666666666666
note_on channel=3 note=56 velocity=67 time=0
note_on channel=3 note=50 velocity=67 time=0
note_on channel=3 note=67 velocity=0 time=0.16666666666666666
note_on channel=3 note=68 velocity=67 time=0
note_on channel=3 note=56 velocity=0 time=0
note_on channel=3 note=50 velocity=0

In [ ]:
mid.ticks_per_beat

480